## Langchain
LangChain is a python based framework to help build LLM-powered applications through the concepts of prompt templates, chains and agents. It is an open-source project [GitHub repository] (https://github.com/hwchase17/langchain) 

In [1]:
from langchain.llms import OpenAI
import os
import api_keys as api

In [2]:
os.environ["OPENAI_API_KEY"] = api.open_ai_api_key
# api keys are saved locally

# your account will be frontloaded with $120 per month for the token usage

## 1.1 Get answers from the Open AI directly

In [3]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
# temp can range from 0 to 2 which means a range of 'deterministic' output to 'non-deterministic' output.
# deterministic means results will not differ at all from 1 prompt

text = "name the top 5 countries with their highest population "
print(llm(text))



1. China - 1.4 billion
2. India - 1.3 billion
3. United States - 329 million
4. Indonesia - 269 million
5. Brazil - 211 million


In [4]:
print(llm("what is the latest month chat gpt model trained on? what version am I talking to?"))



The latest month a chat GPT model has been trained on is April 2021. The version you are talking to is GPT-3.


## 1.2 Using prompt template

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["criteria"],
    template="name the top 5 countries with their highest {criteria} "
)

print(llm(prompt.format(criteria="economy")))



1. United States - $21.44 trillion
2. China - $14.14 trillion
3. Japan - $5.15 trillion
4. Germany - $4.00 trillion
5. India - $2.94 trillion


In [6]:
print(llm(prompt.format(criteria="area")))



1. Russia (17,098,242 km2)
2. Canada (9,984,670 km2)
3. China (9,706,961 km2)
4. United States (9,372,610 km2)
5. Brazil (8,515,767 km2)


## 1.3 Chains: Combine LLMs and prompts

Chains allow us to combine multiple components together to create a single, coherent application. More complex chains can be created by combining multiple chains or componenets together. 

In [7]:
from langchain.chains import LLMChain

In [8]:
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run("economy"))



1. United States - $21.44 trillion
2. China - $14.14 trillion
3. Japan - $5.15 trillion
4. Germany - $4.00 trillion
5. India - $2.94 trillion


## 2.1 Search Google and compare results

Agents use an LLM to determine which actions to take and in what order.  
One of the most common agent is Zero-shot which is based on ReACT framework (a JS library mainatained by Meta) that helps to determine which tool to use based solely on the tool's description.

In [9]:
# !pip install google-search-results

# serpapi is the Google search API
os.environ["SERPAPI_API_KEY"] = api.serp_api_key

In [10]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [16]:
# Load the model
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# load agents
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", 
                         return_intermediate_steps=True, verbose=True)

In [38]:
# Now let's test it out!
response = agent({"input":"""Who is the current leader of USA? what is their age?
           Can you sum up the digits in his age?"""})

print(response["output"])

print([action[0].tool for action in response["intermediate_steps"]])



> Entering new AgentExecutor chain...
 I need to find out who the current leader is and their age.
Action: Search
Action Input: "Current leader of USA"
Observation: Joe Biden
Thought: I need to find out Joe Biden's age.
Action: Search
Action Input: "Joe Biden age"
Observation: 80 years
Thought: I need to sum up the digits in Joe Biden's age.
Action: Calculator
Action Input: 80
Observation: Answer: 80
Thought: I now know the final answer.
Final Answer: Joe Biden is 80 years old and the sum of the digits in his age is 8.

> Finished chain.
Joe Biden is 80 years old and the sum of the digits in his age is 8.
['Search', 'Search', 'Calculator']


## 2.2 Add persona to a prompt template

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

### Getting answers in simple english format

In [45]:
chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that answers to the medical questions in plain english."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("what is Allopurinol")
example_ai = AIMessagePromptTemplate.from_template("Allopurinol is a medicine used to lower levels of uric acid in your blood.")
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [47]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_human, example_ai, human_message_prompt]
)
chain = LLMChain(llm=chat, prompt=chat_prompt)
# get a chat completion from the formatted messages
chain.run("what is Metformin. Explain in maximum 20 words.")

'Metformin is a medication used to treat type 2 diabetes by lowering blood sugar levels and improving insulin sensitivity.'

### Getting answers in complex medical terms

In [56]:
chat = ChatOpenAI(temperature=0)

template = """You are a medical practitioner that answers to the medical questions in difficult medical terms.
Always respond with medical terms. Do not use plain english"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("what is Allopurinol")
example_ai = AIMessagePromptTemplate.from_template("Allopurinol, a xanthine oxidase inhibitor, is a urate-lowering medication that is FDA approved for managing gout, preventing tumor lysis syndrome, and preventing recurrent calcium nephrolithiasis in patients with hyperuricosuria.")
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [61]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_human, example_ai, human_message_prompt]
)
chain = LLMChain(llm=chat, prompt=chat_prompt)
# get a chat completion from the formatted messages
chain.run("what is Metformin? ")

'Metformin is an oral hypoglycemic medication that belongs to the biguanide class of drugs. It is used to treat type 2 diabetes mellitus by reducing glucose production in the liver, decreasing intestinal glucose absorption, and improving insulin sensitivity in peripheral tissues.'